In [ ]:
# Import Neural Network packages from Deep Learning Directory
from DeepLearning import Keras_CNN1D as CNN
from DeepLearning import Keras_FFNN as FFNN
from DeepLearning import Keras_LSTM as LSTM
from DeepLearning import Keras_MLP as MLP
from DeepLearning import ModularNN as mNN
from DeepLearning import Keras_Perceptron as Perceptron
from DeepLearning import Keras_RBF as RBF
from DeepLearning import Keras_RNN as RNN 
from DeepLearning import Keras_SNN as SNN

In [ ]:
# Import Technical Application and Wavelet denoising functions
from pre_processing import ta_features as ta
from pre_processing import wavelet_fx as denoise
import numpy as np

In [40]:
# Import neural network libraries
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, BatchNormalization
from keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor

In [ ]:
# Import Graph libraries
import matplotlib.pyplot as plt

In [ ]:
# Import basic libraries
import pandas as pd
import numpy as np
import io
import os
import mglearn
import sys
import requests

def import_dataframes_from_folder(folder_path):
    """
    Imports all CSV files from a folder with separator='\t' and returns a dictionary of dataframes.

    Parameters:
    folder_path (str): The path to the folder containing the CSV files.

    Returns:
    dict: A dictionary mapping CSV file names to their corresponding dataframes.
    """
    dataframes = {}
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            filepath = os.path.join(folder_path, filename)
            df_name = filename[:-4]  # remove .csv extension from filename
            df = pd.read_csv(filepath, sep='\t')
            dataframes[df_name] = df
    return dataframes

data_daily = 'Historical_Data/Daily'
data_M1 = 'Historical_Data/M1_2019_01_01__2021_01_01_0000'
data_M5 = 'Historical_Data/M5_2019_01_01__2021_01_01_0000'
data_M10 = 'Historical_Data/M10_2019_01_01__2021_01_01_0000'
data_M15 = 'Historical_Data/M15_2019_06_15__2021_06_15_0000'
data_M30 = 'Historical_Data/M30_2018_01_01__2022_01_01_0000'
data_1H = 'Historical_Data/1H_2018_06_01__2022_06_01_0000'
data_6H = 'Historical_Data/6H_2014_01_01__2022_01_01_0000'
data_12H = 'Historical_Data/12H_2014_01_01__2022_01_01_0000'

dataframes_daily = import_dataframes_from_folder(data_daily)
dataframes_M1 = import_dataframes_from_folder(data_M1)
dataframes_M5 = import_dataframes_from_folder(data_M5)
dataframes_M10 = import_dataframes_from_folder(data_M10)
dataframes_M15 = import_dataframes_from_folder(data_M15)
dataframes_M30 = import_dataframes_from_folder(data_M30)
dataframes_H1 = import_dataframes_from_folder(data_1H)
dataframes_H6 = import_dataframes_from_folder(data_6H)
dataframes_H12 = import_dataframes_from_folder(data_12H)

In [ ]:
data = dataframes_daily['EURCHF_Daily']

data = data.rename(columns = {'<DATE>':'Date','<OPEN>':'open', '<HIGH>':'high', '<LOW>':'low', '<CLOSE>':'close','<TICKVOL>':'tickvol','<VOL>':'volume','<SPREAD>':'spread'})

data = ta.TA_APPLICATION(data)

data

data['price_change'] = data['close'].diff()
volatility = np.std(data['price_change'])
print("Market volatility: ", volatility)

In [38]:
# print(dataframes.keys())
# data = dataframes['EURCHF_Daily']
# data

processed_data = dataframes_daily['EURCHF_Daily']

processed_data = processed_data.rename(columns = {'<DATE>':'Date','<OPEN>':'Open', '<HIGH>':'High', '<LOW>':'Low', '<CLOSE>':'Close','<TICKVOL>':'tickvol','<VOL>':'volume','<SPREAD>':'spread'})

processed_data['open_wave'],processed_data['high_wave'],processed_data['low_wave'],\
processed_data['close_wave']=denoise.denoising([processed_data['Open'],processed_data['High'],processed_data['Low'],processed_data['Close']])

processed_data = processed_data[['open_wave', 'high_wave', 'low_wave', 'close_wave', 'tickvol', 'volume', 'spread']]
processed_data = ta.TA_WAVE_APPLICATION(processed_data)

In [ ]:
# Set up window for rolling data
def window_data(data, window, feature_col_numbers, target_col_number):
    X = []
    y = []
    for i in range(len(data) - window - 1):
        features = data.iloc[i:(i + window), feature_col_numbers]
        target = data.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

# Window settings for tuning
window_size = 1
# feature_columns = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32]
feature_columns = [0,1,2,3,18,22,21,22,23,26,30,31,32]
# feature_columns = [0,1,2,3]
target_column = 3
X, y = window_data(processed_data, window_size, feature_columns, target_column)

# Print target variable and selected features
print(f"Target variable: {processed_data.columns[target_column]}")
print("Selected features:")
for col in feature_columns:
    print(processed_data.columns[col])
    
# Use 80% of the data for training and the remaineder for testing
split = int(0.8 * len(X))
X_train = X[: split]
X_test = X[split:]
y_train = y[: split]
y_test = y[split:]

# Use the MinMaxScaler to scale data between 0 and 1.
scaler = MinMaxScaler()
scaler.fit(X_train.reshape(-1, len(feature_columns)))
X_train = scaler.transform(X_train.reshape(-1, len(feature_columns)))
X_test = scaler.transform(X_test.reshape(-1, len(feature_columns)))
scaler.fit(y_train)
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)


from sklearn.impute import SimpleImputer

# Fill NaN values with mean
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

print(np.isnan(X_train).any())
print(np.isnan(X_test).any())
print(np.isinf(X_train).any())
print(np.isinf(X_test).any())

X_train.shape, y_test.shape

In [45]:
# Define the function to create the model - > /DeepLearning/Keras_LSTM
def build_model_lstm(hidden_layers=2, units=32, dropout_rate=0.1, output_units=1, optimizer='adam', loss='binary_crossentropy'):
    
        model = Sequential()
        
        # Input layer
        model.add(LSTM(units=units, input_shape=(X_train.shape[1], 1), return_sequences=True))
        #model.add(BatchNormalization())
        model.add(Dropout(rate=dropout_rate))
        
        # Hidden layers
        for i in range(hidden_layers - 1):
            model.add(LSTM(units=units, return_sequences=True))
            #model.add(BatchNormalization())
            model.add(Dropout(rate=dropout_rate))
        
        # Output layer
        model.add(LSTM(units=output_units))
        model.add(Dense(units=output_units))
        
        # Compile the model
        model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
        return model
    
# Define the function to create the model - > /DeepLearning/Keras_FFNN
def build_model_ffnn(hidden_layers=2, units=32, dropout_rate=0.1, output_units=1, optimizer='adam', loss='binary_crossentropy'):
        
        model = Sequential()
        
        # Input layer
        model.add(Dense(units=units, input_shape=(X_train.shape[1], 1)))
        #model.add(BatchNormalization())
        model.add(Dropout(rate=dropout_rate))
        
        # Hidden layers
        for i in range(self.hidden_layers - 1):
            model.add(Dense(units=units))
            #model.add(BatchNormalization())
            model.add(Dropout(rate=dropout_rate))
        
        # Output layer
        model.add(Dense(units=output_units))
        
        # Compile the model
        model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
        return model
                      
# Define the function to create the model - > /DeepLearning/Keras_MLP           
def build_model_mlp(hidden_layers=2, units=32, dropout_rate=0.1, output_units=1, optimizer='adam', loss='binary_crossentropy'):
                      
        model = Sequential()

        # Input layer
        model.add(Dense(units=hidden_layers[0], input_shape=(X_train.shape[1], 1)))
        #model.add(BatchNormalization())              
        model.add(Activation(self.activation))

        # Hidden layers
        for layer_size in hidden_layers[1:]:
            model.add(Dense(units=units))
            #model.add(BatchNormalization())          
            model.add(Activation(self.activation))

        # Output layer
        model.add(Dense(units=output_units))
        #model.add(BatchNormalization())              
        model.add(Activation('softmax'))

        # Compile the model
        model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
        return model

# Define the function to create the model - > /DeepLearning/Keras_Perceptron                                 
def build_model_perceptron(hidden_layers=2, units=32, dropout_rate=0.1, output_units=1, optimizer='adam', loss='binary_crossentropy'):
                      
        model = Sequential()

        # Input layer
        model.add(Dense(units=output_units, input_shape=(X_train.shape[1], 1)))
        #model.add(BatchNormalization())               
        model.add(Activation('sigmoid'))

        # Compile the model
        model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
        return model
                      
# Define the function to create the model - > /DeepLearning/Keras_RNN                                                      
def build_model_rnn(hidden_layers=2, units=32, dropout_rate=0.1, output_units=1, optimizer='adam', loss='binary_crossentropy'):
        model = Sequential()
        
        # Input layer
        model.add(SimpleRNN(units=units, return_sequences=True, input_shape=(X_train.shape[1], 1)))
        # model.add(BatchNormalization())
        model.add(Dropout(rate=dropout_rate))
        
        # Hidden layers
        for i in range(hidden_layers - 1):
            model.add(SimpleRNN(units=self.units, return_sequences=True))
            # model.add(BatchNormalization())
            model.add(Dropout(rate=dropout_rate))
        
        # Output layer
        model.add(SimpleRNN(units=output_units))
        model.add(Dense(units=output_units))
        
        # Compile the model
        model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
        return model  

# Define the function to create the model - > /DeepLearning/ModularNN                                                                       
def build_model_ModularNN(hidden_layers=2, units=32, dropout_rate=0.1, output_units=1, optimizer='adam', loss='binary_crossentropy'):
        inputs = Input(shape=(X_train.shape[1], 1))
        x = inputs

        for units, activation, dropout in hidden_layers:
            x = Dense(units, activation=activation)(x)
            if dropout > 0:
                x = Dropout(dropout)(x)

        outputs = Dense(output_shape, activation='softmax')(x)
        model = Model(inputs=inputs, outputs=outputs)
        model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

        return model

In [47]:
# Create the KerasClassifier object
model = KerasClassifier(build_fn=build_model_lstm, epochs=10, batch_size=32, verbose=0)

# Define the hyperparameters to search over
param_grid_classifier = {
    'optimizer': ['adam', 'sgd', 'rmsprop'],
    'hidden_layers': np.arange(1, 4),
    'dropout_rate': np.linspace(0, 0.5, 6),
    'units': np.arange(1, 10),
    'loss': ['binary_crossentropy', 'mse', 'categorical_crossentropy']
}

# Create the RandomizedSearchCV object
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_grid_classifier,
    n_iter=10,
    cv=3,
    verbose=2,
    random_state=42
)

# Fit the model using the RandomizedSearchCV object
random_search.fit(X_train, y_train)

# Print the best hyperparameters
print(random_search.best_params_)

# Define parameter grid
param_grid_regressor = {
    'optimizer': ['adam', 'sgd', 'rmsprop'],
    'hidden_layers': [1, 2],
    'units': [32, 64],
    'dropout_rate': [0.1, 0.2],
    'batch_size': [32, 64],
    'epochs': [10, 20],
    'loss': ['binary_crossentropy', 'mse','categorical_crossentropy']
}

# # Create KerasRegressor object for Grid Search
# model = KerasRegressor(build_fn=build_model_lstm, verbose=0)

# # Create Grid Search object
# grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1)

# # Fit Grid Search object to training data
# grid_search.fit(X_train, y_train)

# # Print best hyperparameters and R^2 score
# print("Best hyperparameters: ", grid_search.best_params_)
# print("R^2 score: ", grid_search.best_score_)

C:\Users\Adrian\AppData\Local\Temp\ipykernel_5716\4156450443.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=build_model_lstm, epochs=10, batch_size=32, verbose=0)


Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END dropout_rate=0.4, hidden_layers=2, loss=categorical_crossentropy, optimizer=rmsprop, units=2; total time=   7.2s
[CV] END dropout_rate=0.4, hidden_layers=2, loss=categorical_crossentropy, optimizer=rmsprop, units=2; total time=   8.0s
[CV] END dropout_rate=0.4, hidden_layers=2, loss=categorical_crossentropy, optimizer=rmsprop, units=2; total time=   7.1s
[CV] END dropout_rate=0.30000000000000004, hidden_layers=2, loss=mse, optimizer=rmsprop, units=6; total time=   7.1s
[CV] END dropout_rate=0.30000000000000004, hidden_layers=2, loss=mse, optimizer=rmsprop, units=6; total time=   7.2s
[CV] END dropout_rate=0.30000000000000004, hidden_layers=2, loss=mse, optimizer=rmsprop, units=6; total time=   8.1s
[CV] END dropout_rate=0.5, hidden_layers=1, loss=categorical_crossentropy, optimizer=rmsprop, units=8; total time=   5.0s
[CV] END dropout_rate=0.5, hidden_layers=1, loss=categorical_crossentropy, optimizer=rmsprop, units=